In [1]:
#%pip install --upgrade langchain langsmith langgraph langchain_openai openai python-dotenv javascript
# run this in terminal: python3 -m javascript --install @near-lake/primitives
%reload_ext autoreload
%autoreload 2

In [2]:
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv('.env')

# Set model variables
OPENAI_BASE_URL = "https://api.openai.com/v1"
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_ORGANIZATION = os.getenv("OPENAI_ORGANIZATION")


os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "Indexer Agent Local"


# Execute the Agent

In [33]:
from langchain_core.messages import HumanMessage
from graph.trivial_agent_graph import IndexerAgentGraphBuilder
from agents.IndexerAgent import indexer_agent_model, tool_executor
from tools.JavaScriptRunner import tool_js_on_block, tool_js_on_block_schema
from agents.BlockExtractorAgent import block_extractor_agent_model
import json

In [34]:


inputs = {
    "messages": [
        HumanMessage(
            content="Write code to extract all FunctionCalls where receiverId is 'app.nearcrowd.near'. For each function call, I need signerId, block height, receiptId, block datetime, methodName and args that are decoded from base64-encoded JSON. Only output JS code and no other extra information. I only need actions for successful receipts.",
        ),
        HumanMessage(
            content="You can use various block heights like 119688211, 119688186, 119688185 to understand the structure of the blocks by running JavaScript code using Run_Javascript_On_Block_Schema",
        )
    ]
}

# Define the tools that the agent will use
tools = [tool_js_on_block, tool_js_on_block_schema]
model = block_extractor_agent_model(tools)

# print(model.invoke(input=inputs['messages']))

builder = IndexerAgentGraphBuilder(model, tool_executor(tools))
app = builder.graph()

result = app.with_config({"run_name": "IndexerAgentPavel"}).invoke(inputs)

# for output in app.with_config({"run_name": "IndexerAgentPavel"}).stream(inputs):
#     # stream() yields dictionaries with output keyed by node name
#     for key, value in output.items():
#         print(f"----------------- Output from node '{key}':")
#         print(json.dumps(value, indent=2))
#     print("\n---\n")


In [35]:
result['messages'][-1]

AIMessage(content="Sure, here is the JavaScript code that you can use to extract the required information:\n\n```javascript\n// Importing necessary libraries\nconst atob = require('atob');\n\n// Function to decode base64-encoded JSON\nfunction decodeArgs(args) {\n    let decodedArgs = atob(args);\n    return JSON.parse(decodedArgs);\n}\n\n// Function to extract required information\nfunction extractFunctionCalls(block) {\n    let result = [];\n    let receipts = block.receipts();\n    let header = block.header();\n    let actions = block.actions();\n\n    for (let i = 0; i < receipts.length; i++) {\n        if (receipts[i].status.SuccessValue && receipts[i].receiverId === 'app.nearcrowd.near') {\n            for (let j = 0; j < actions.length; j++) {\n                if (actions[j].receiptId === receipts[i].receiptId) {\n                    let operations = actions[j].operations;\n                    for (let k = 0; k < operations.length; k++) {\n                        if (operations[

In [ ]:

from tools.JavaScriptRunner import run_js_on_block_only_schema, run_js_on_block
code = """
return block.actions()
"""
print(json.dumps(run_js_on_block_only_schema(119688212, 'return block.actions()')))

In [18]:
from agents.BlockExtractorAgent import block_extractor_agent_model

tools = []
model = block_extractor_agent_model(tools)

inputs = {
    "messages": [
        HumanMessage(
            content="Write code to extract all FunctionCalls where receiverId is 'app.nearcrowd.near'. For each function call, I need signerId, block height, receiptId, block datetime, methodName and args that are decoded from base64-encoded JSON. Only output JS code and no other extra information. I only need actions for successful receipts.",
        )
    ]
}

res = model.invoke(inputs['messages'])

AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_hjboT7SQJojQmCMmZWKzrjM1', 'function': {'arguments': '{\n"js": `\nfunction extractFunctionCalls(block) {\n    const actions = block.actions();\n    const receipts = block.receipts();\n    const header = block.header();\n    const successfulReceipts = receipts.filter(receipt => receipt.status.SuccessValue);\n    const functionCalls = [];\n\n    successfulReceipts.forEach(receipt => {\n        actions.forEach(action => {\n            if (action.receiptId === receipt.receiptId && action.receiverId === \'app.nearcrowd.near\') {\n                action.operations.forEach(operation => {\n                    if (operation.FunctionCall) {\n                        const functionCall = operation.FunctionCall;\n                        const args = JSON.parse(atob(functionCall.args));\n                        functionCalls.push({\n                            signerId: action.signerId,\n                            bloc

In [31]:
json.loads(res.additional_kwargs["tool_calls"][0]['function']['arguments'])

JSONDecodeError: Expecting value: line 2 column 7 (char 8)

In [16]:
res.content

"Here is the JavaScript code that will extract all FunctionCalls where receiverId is 'app.nearcrowd.near'. It will also decode the base64-encoded JSON args.\n\n```javascript\n// Assuming block is the block object\nfunction extractFunctionCalls(block) {\n    const actions = block.actions();\n    const receipts = block.receipts();\n    const header = block.header();\n    const blockHeight = header.height;\n    const blockDatetime = new Date(parseInt(header.timestampNanosec) / 1000000);\n\n    const successfulReceipts = receipts.filter(receipt => receipt.status.SuccessValue !== undefined);\n    const successfulReceiptIds = successfulReceipts.map(receipt => receipt.receiptId);\n\n    const functionCalls = actions.filter(action => {\n        return action.receiverId === 'app.nearcrowd.near' &&\n            successfulReceiptIds.includes(action.receiptId) &&\n            action.operations.some(operation => operation.FunctionCall !== undefined);\n    });\n\n    return functionCalls.map(functio